In [1]:
from corpus.opensubsdata import OpensubsData
from tqdm import tqdm
import re

In [2]:
data = OpensubsData('data')

Loading OpenSubtitles conversations in data.


Skipping file data\OpenSubtitles\en\Action\2003\602_152466_207871_batoru_rowaiaru_ii_rekuiemu.xml.gz with errors.


Skipping file data\OpenSubtitles\en\Action\2004\59_84873_113518_appurushdo.xml.gz with errors.


Skipping file data\OpenSubtitles\en\Comedy\2003\529_124078_171007_how_to_lose_a_guy_in_10_days.xml.gz with errors.


Skipping file data\OpenSubtitles\en\Comedy\2004\2480_226704_299940_little_black_book.xml.gz with errors.


...

Skipping file data\OpenSubtitles\en\Drama\2003\1723_68784_89159_big_fish.xml.gz with errors.


Skipping file data\OpenSubtitles\en\Drama\2004\146_206647_272090_eternal_sunshine_of_the_spotless_mind.xml.gz with errors.


Skipping file data\OpenSubtitles\en\Family\2001\3935_19508_22105_cats__dogs.xml.gz with errors.


Skipping file data\OpenSubtitles\en\Horror\1922\1166_134135_184270_nosferatu_eine_symphonie_des_grauens.xml.gz with errors.


OpenSubtitles data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2029/2029 [02:25<00:00, 13.98it/s]


In [3]:
len(data.getConversations())

1468840

In [4]:
def normalize(text):
#     text = re.sub('[0-9]', '[NUMBER]')
    text = re.sub('[^a-zA-Z ]', '', text)
    words = text.split()
    return words

In [5]:
conversations = []
for conversation in tqdm(data.getConversations()):
    q = conversation['lines'][0]['text']
    a = conversation['lines'][1]['text']
    conversations.append([normalize(q), normalize(a)])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1468840/1468840 [00:14<00:00, 102060.75it/s]


In [6]:
print(conversations[5004])

[['why', 'it', 'won', 't', 'work'], ['this', 'is', 'the', 'northern', 'hemisphere', 'correct']]


In [7]:
from collections import Counter

In [8]:
def make_dict(conversations):
    vocab = Counter()
    for conversation in conversations:
        for sentence in conversation:
            vocab.update(sentence)

    unk_word_cutoff_freq = 10
    sorted_words = ['_'] + sorted([word for word in vocab.keys() 
                                   if vocab[word] > unk_word_cutoff_freq])
    index_to_word = dict(enumerate(sorted_words))
    word_to_index = dict((v, k) for k,v in index_to_word.items())
    
    vocab_size = len(index_to_word)
    index_to_word[vocab_size] = 'UNK'
    word_to_index['UNK'] = vocab_size
    
    return index_to_word, word_to_index

In [9]:
def encode_conversations(conversations, word_to_index):

    """
    Input//  
    conversations = List of conversation
        conversation = [sentence1, sentence2]
        sentence = List of String
    word_to_index = Dictionary, key = word(string), value = index(int)
    
    Output//
    encoded_conversations = List of encoded_conversation
        enconded_conversation = [encoded_sentence1, encoded_sentence2]
        encoded_sentence = List of Int, Length == Maximum Length of Sentence in Training Data
    """
    
    encoded_conversations = []
    max_sentence_len = max(len(sentence) for conversation in conversations 
                                           for sentence in conversation)
    
    def encode_sentence(sentence, word_to_index):
        encoded_sentence = []
        for word in sentence:
            if word in word_to_index.keys():
                encoded_sentence.append(word_to_index[word])
            else:
                encoded_sentence.append(word_to_index['UNK'])
        # Sentence Length == max_len
        encoded_sentence += [0] * (max_sentence_len - len(encoded_sentence))
        return encoded_sentence

    for conversation in conversations:
        encoded_conversations.append([encode_sentence(sentence, word_to_index) 
                                          for sentence in conversation])
    return encoded_conversations

In [10]:
index_to_word, word_to_index = make_dict(conversations)

In [11]:
len(index_to_word)

31555

In [12]:
len(word_to_index)

31555

In [13]:
vocab_size = len(index_to_word)

In [14]:
data = encode_conversations(conversations, word_to_index)

In [15]:
print(data[1])

[[12778, 21009, 4377, 12407, 23961, 1628, 28417, 26235, 28326, 30992, 17682, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [17682, 14920, 3831, 12907, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [16]:
import tensorflow as tf
from tensorflow.python.framework import ops

In [17]:
ops.reset_default_graph()

In [18]:
sentence_length = len(data[0][0])
batch_size = 32
embedding_dim = 100

In [19]:
encode_input = [tf.placeholder(tf.int32, shape=(None,), name = "ei_%i" %i) 
                    for i in range(sentence_length)]
labels = [tf.placeholder(tf.int32, shape=(None,), name = "l_%i" %i)
                    for i in range(sentence_length)]
decode_input = [tf.zeros_like(encode_input[0], dtype=tf.int32, name="GO")] + labels[:-1]

In [20]:
cells = [tf.contrib.rnn.GRUCell(embedding_dim) for i in range(1)]
# stacked_gru = tf.contrib.rnn.MultiRNNCell(cells)
stacked_gru = cells[0] # At now, Use Just One Cell

In [21]:
seq2seq_config = {
    'encoder_inputs':encode_input,
    'decoder_inputs':decode_input,
    'cell':stacked_gru,
    'num_encoder_symbols':vocab_size,
    'num_decoder_symbols':vocab_size,
    'embedding_size':embedding_dim
    }

In [22]:
with tf.variable_scope("decoders") as scope:
    decode_outputs, _ = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(**seq2seq_config, 
                                                                        feed_previous=False)
    scope.reuse_variables()
    decode_outputs_test, _ = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(**seq2seq_config,
                                                                             feed_previous=True)

In [23]:
loss_weights = [tf.ones_like(l, dtype=tf.float32) for l in labels]
loss = tf.contrib.legacy_seq2seq.sequence_loss(decode_outputs, labels, loss_weights, vocab_size)
optimizer = tf.train.AdamOptimizer(1e-4)
train_op = optimizer.minimize(loss)

In [24]:
sess = tf.InteractiveSession()

In [25]:
sess.run(tf.global_variables_initializer())

In [26]:
import numpy as np

In [27]:
class BatchGenerator:
    
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.iter = self.make_random_iter()

    def make_random_iter(self):
        splits = np.arange(self.batch_size, len(self.data), self.batch_size)
        it = np.split(np.random.permutation(range(len(self.data))), splits)[:-1]
        return iter(it)
        
    def next_batch(self):
        try:
            idxs = next(self.iter)
        except StopIteration:
            self.iter = self.make_random_iter()
            idxs = next(self.iter)
        
        X = [self.data[i][0] for i in idxs]
        Y = [self.data[i][1] for i in idxs]
        X = np.array(X).T
        Y = np.array(Y).T
        return X, Y

In [28]:
def get_feed(X, Y):
    feed_dict = {encode_input[t]: X[t] for t in range(sentence_length)}
    feed_dict.update({labels[t]: Y[t] for t in range(sentence_length)})
    return feed_dict

def train_batch(data_iter):
    X, Y = data_iter.next_batch()
    feed_dict = get_feed(X, Y)
    _, out = sess.run([train_op, loss], feed_dict)
    return out

def get_eval_batch_data(data_iter):
    X, Y = data_iter.next_batch()
    train_iter = BatchGenerator(train_data, 32)
    val_iter = BatchGenerator(val_data, 10)
    test_iter = BatchGenerator(test_data, 10)
    feed_dict = get_feed(X, Y)
    all_output = sess.run([loss] + decode_outputs_test, feed_dict)
    eval_loss = all_output[0]
    decode_output = np.array(all_output[1:]).transpose([1,0,2])
    return eval_loss, decode_output, X, Y

def eval_batch(data_iter, num_batches):
    losses = []
    predict_loss = []
    for i in range(num_batches):
        eval_loss, output, X, Y = get_eval_batch_data(data_iter)
        losses.append(eval_loss)

        for index in range(len(output)):
            real = Y.T[index]
            predict = np.argmax(output, axis = 2)[index]
            predict_loss.append(all(real==predict))
    return np.mean(losses), np.mean(predict_loss)

In [29]:
num_instance = len(data)
print(num_instance)

1468840


In [30]:
train_data = data[:int(num_instance * 0.6)]
val_data = data[int(num_instance * 0.6) : int(num_instance * 0.9)]
test_data = data[int(num_instance * 0.9):]

In [ ]:
train_iter = BatchGenerator(train_data, 32)
val_iter = BatchGenerator(train_data, 10)
test_iter = BatchGenerator(train_data, 10)

In [ ]:
for i in range(10000):
    try:
        train_batch(train_iter)
        if i % 5 == 0:
            val_loss, val_predict = eval_batch(val_iter, 10)
            train_loss, train_predict = eval_batch(train_iter, 10)
            
            print ('iteration : %f' %(i))
            print ('val loss   : %f' %(val_loss))
            print ('train loss : %f' %(train_loss))
#             print
#             sys.stdout.flush()
#             if i % 2000 == 0:
#                 show_text('train')

    except KeyboardInterrupt:
        print ("interrupted by user")
        break

iteration : 0.000000
val loss   : 10.355874
train loss : 10.355784
iteration : 5.000000
val loss   : 10.347193
train loss : 10.346895
iteration : 10.000000
val loss   : 10.336492
train loss : 10.336823
iteration : 15.000000
val loss   : 10.322632
train loss : 10.323482
iteration : 5705.000000
val loss   : 1.163440
train loss : 1.108397
iteration : 5710.000000
val loss   : 1.041192
train loss : 1.090889
iteration : 5715.000000
val loss   : 1.100498
train loss : 1.054541
iteration : 5720.000000
val loss   : 1.105267
train loss : 1.099669
iteration : 5725.000000
val loss   : 1.104627
train loss : 1.074680
iteration : 5730.000000
val loss   : 1.051386
train loss : 1.054379
iteration : 5735.000000
val loss   : 1.121879
train loss : 1.043492
iteration : 5740.000000
val loss   : 1.165042
train loss : 1.088795
iteration : 5745.000000
val loss   : 1.115464
train loss : 1.079772
iteration : 5750.000000
val loss   : 1.103851
train loss : 1.072879
iteration : 5755.000000
val loss   : 1.121076
trai

In [33]:
def show_text(data_type):
    if data_type == 'train':
        eval_loss, output, X, Y = get_eval_batch_data(train_iter)
    elif data_type == 'test':
        eval_loss, output, X, Y = get_eval_batch_data(test_iter)

    for index in range(len(output)):
        input_text = " ".join([index_to_word[p] for p in X.T[index]])
        label_text = " ".join([index_to_word[l] for l in Y.T[index]])
        predicted_text = " ".join([index_to_word[l] for l in np.argmax(output, axis = 2)[index]])

        print (input_text.split("_")[0].ljust(40))
        print ("".join(label_text).split("_")[0].ljust(17))
        print ("".join(predicted_text).split("_")[0].ljust(17), '\n')

In [37]:
show_text('test')

                                        
here we go here we go go UNK go megget 
                  

go ahead surprise me                    
sorry chief that only gets you halfway there 
i t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t 

lf your child s school has old ass books 
and brand new metal detectors let me hear you say that ain t right 
i m t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t 

really i m serious                      
anyway why did she faint 
i t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t 

i believe a light that shines on you    
will shine on you forever 
i m t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t 

but don t put your hand down there      
i didn t mean    
i m t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t 

wives and family and stuff yeah i mean not children 
but their wives